<a href="https://colab.research.google.com/github/zeehayat/data-science/blob/main/notebookda61690cc3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'wb-poverty:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F14439%2F19370%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240929%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240929T151437Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5028da210356b6e7347dda42b8a1dc199ba8c6b70ec3c17661c164ed4a6f8b8f2a896f173213a23e3ded4b52e4387dc0e6d46fe6e013108a82c7b0273b1b4d92717385dd0a65c642ed73078141c94095cf44ae754ad7db9c310834e8b942bb002e5be63157d29eef14daaa8bc822f3d4c7288f73f2413a4a0669bc75b24c8b57343d1af56754a73f7f63e8071debb449586e1a8a1cac004fd43c8cee050d7b333effceda2c559f6e5b108bf46b515fcbae489571da6995ca37db57cd24807d28909ab547c3f1c382bd755a9e182d9ccd67e005e9b5ed07a3ac615790eec26a1b7a64245f0c2a07c067e86652b8c580580b88f8d12ff36f22daf44330b79efc33'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 110037 bytes downloaded
Downloaded and uncompressed: wb-poverty
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wb-poverty/wb_subnational_poverty_data.csv
/kaggle/input/wb-poverty/subnational_poverty_footnote.csv
/kaggle/input/wb-poverty/subnational_poverty_country.csv


In [3]:
df=pd.read_csv('/kaggle/input/wb-poverty/wb_subnational_poverty_data.csv')
df.head()

,indicator_name,indicator_code,country_name,country_code,value,year
0,Poverty headcount ratio at national poverty li...,SI.POV.NAHC,Mozambique,MOZ,69.4,1996-01-01
1,Poverty headcount ratio at national poverty li...,SI.POV.NAHC,"Mozambique, Cabo Delgado",MOZ_Cabo_Delgado_MZ.CD_2112.1_MOZ001,57.4,1996-01-01
2,Poverty headcount ratio at national poverty li...,SI.POV.NAHC,"Mozambique, Gaza",MOZ_Gaza_MZ.GA_2113.1_MOZ002,64.6,1996-01-01
3,Poverty headcount ratio at national poverty li...,SI.POV.NAHC,"Mozambique, Inhambane",MOZ_Inhambane_MZ.IN_2114.1_MOZ003,82.6,1996-01-01
4,Poverty headcount ratio at national poverty li...,SI.POV.NAHC,"Mozambique, Manica",MOZ_Manica_MZ.MN_2115.1_MOZ004,62.6,1996-01-01


In [5]:
# Group by Country and Year

df.groupby(['country_name', 'year']).sum()

indicator_name  \
country_name         year                                                            
Afghanistan          2007-01-01  Poverty headcount ratio at national poverty li...   
                     2011-01-01  Poverty headcount ratio at national poverty li...   
Afghanistan, Central 2007-01-01  Poverty headcount ratio at national poverty li...   
                     2011-01-01  Poverty headcount ratio at national poverty li...   
Afghanistan, East    2007-01-01  Poverty headcount ratio at national poverty li...   
...                                                                            ...   
Zambia, Northern     2010-01-01  Poverty headcount ratio at national poverty li...   
Zambia, Southern     2006-01-01  Poverty headcount ratio at national poverty li...   
                     2010-01-01  Poverty headcount ratio at national poverty li...   
Zambia, Western      2006-01-01  Poverty headcount ratio at national poverty li...   
                     2010-01-01  Poverty headcount ratio at national poverty li...   

                                indicator_code  \
country_name         year                        
Afghanistan          2007-01-01    SI.POV.NAHC   
                     2011-01-01    SI.POV.NAHC   
Afghanistan, Central 2007-01-01    SI.POV.NAHC   
                     2011-01-01    SI.POV.NAHC   
Afghanistan, East    2007-01-01    SI.POV.NAHC   
...                                        ...   
Zambia, Northern     2010-01-01    SI.POV.NAHC   
Zambia, Southern     2006-01-01    SI.POV.NAHC   
                     2010-01-01    SI.POV.NAHC   
Zambia, Western      2006-01-01    SI.POV.NAHC   
                     2010-01-01    SI.POV.NAHC   

                                                   country_code      value  
country_name         year                                                   
Afghanistan          2007-01-01                             AFG  36.300000  
                     2011-01-01                             AFG  35.800000  
Afghanistan, Central 2007-01-01              AFG_Central_AGG001  29.004498  
                     2011-01-01              AFG_Central_AGG001  27.979639  
Afghanistan, East    2007-01-01                 AFG_East_AGG002  45.355446  
...                                                         ...        ...  
Zambia, Northern     2010-01-01  ZMB_Northern_ZM.NO_3432_ZMB006  75.000000  
Zambia, Southern     2006-01-01  ZMB_Southern_ZM.SO_3433_ZMB008   7.000000  
                     2010-01-01  ZMB_Southern_ZM.SO_3433_ZMB008  67.900000  
Zambia, Western      2006-01-01   ZMB_Western_ZM.WE_3434_ZMB009  83.300000  
                     2010-01-01   ZMB_Western_ZM.WE_3434_ZMB009  80.400000  

[2727 rows x 4 columns]

In [11]:
# Extract the country name by splitting on ',' and taking the first element
df['country_name'] = df['country_name'].astype(str).str.split(',').str[0]

# Group data by `country_name` and `year`, calculate the mean of `value`
df_grouped = df.groupby(['country_name', 'year'])['value'].mean().reset_index()

# Sort the grouped dataframe by `country_name` and `year`
df_grouped = df_grouped.sort_values(['country_name', 'year'])

# Display the first 5 rows of the grouped dataframe
print(df_grouped.head().to_markdown(index=False, numalign="left", stralign="left"))
# Group data by `year`, calculate the mean of `value`
df_year = df_grouped.groupby(['year'])['value'].mean().reset_index()

# Sort the grouped dataframe by `year`
df_year = df_year.sort_values(['year'])

# Display the first 5 rows of the grouped dataframe
print(df_year.head().to_markdown(index=False, numalign="left", stralign="left"))
# Pivot the dataframe
df_pivoted = df_grouped.pivot(index='country_name', columns='year', values='value')

# Display the first 5 rows
print(df_pivoted.head().to_markdown(numalign="left", stralign="left"))
# Pivot the dataframe
df_pivoted = df_grouped.pivot(index='country_name', columns='year', values='value')

# Calculate the change between the first and last available year for each country
df_pivoted['change'] = df_pivoted.ffill(axis=1).iloc[:, -1] - df_pivoted.bfill(axis=1).iloc[:, 0]

# Sort by `change` in descending order
df_pivoted_sorted = df_pivoted.sort_values('change', ascending=False)

# Display the first 5 rows
df_pivoted_sorted.head() #.to_markdown(numalign="left", stralign="left"))

| country_name   | year       | value   |
|:---------------|:-----------|:--------|
| Afghanistan    | 2007-01-01 | 37.5549 |
| Afghanistan    | 2011-01-01 | 36.8459 |
| Albania        | 2008-01-01 | 14.28   |
| Albania        | 2012-01-01 | 14.46   |
| Armenia        | 2012-01-01 | 32.05   |
| year       | value   |
|:-----------|:--------|
| 1996-01-01 | 68.9833 |
| 2000-01-01 | 32.047  |
| 2001-01-01 | 33.0614 |
| 2002-01-01 | 41.414  |
| 2003-01-01 | 45.8269 |
| country_name   | 1996-01-01   | 2000-01-01   | 2001-01-01   | 2002-01-01   | 2003-01-01   | 2004-01-01   | 2005-01-01   | 2006-01-01   | 2007-01-01   | 2008-01-01   | 2009-01-01   | 2010-01-01   | 2011-01-01   | 2012-01-01   | 2013-01-01   |
|:---------------|:-------------|:-------------|:-------------|:-------------|:-------------|:-------------|:-------------|:-------------|:-------------|:-------------|:-------------|:-------------|:-------------|:-------------|:-------------|
| Afghanistan    | nan          | nan      

year,1996-01-01,2000-01-01,2001-01-01,2002-01-01,2003-01-01,2004-01-01,2005-01-01,2006-01-01,2007-01-01,2008-01-01,2009-01-01,2010-01-01,2011-01-01,2012-01-01,2013-01-01,change
country_name,,,,,,,,,,,,,,,,
Zambia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.390000,NaN,NaN,NaN,62.88,NaN,NaN,NaN,4.490000
Kyrgyz Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.855556,NaN,37.666667,36.100000,36.788889,3.933333
Guinea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.477778,NaN,NaN,NaN,NaN,56.688889,NaN,3.211111
Egypt,NaN,15.55,NaN,NaN,NaN,NaN,18.116667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.566667
Guatemala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.818182,NaN,NaN,NaN,NaN,56.109091,NaN,NaN,2.290909


In [15]:
# replace na with 0
df_filled = df_pivoted_sorted.fillna(0)
df_filled.head()

year,1996-01-01,2000-01-01,2001-01-01,2002-01-01,2003-01-01,2004-01-01,2005-01-01,2006-01-01,2007-01-01,2008-01-01,2009-01-01,2010-01-01,2011-01-01,2012-01-01,2013-01-01,change
country_name,,,,,,,,,,,,,,,,
Zambia,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,58.390000,0.000000,0.0,0.000000,62.88,0.000000,0.000000,0.000000,4.490000
Kyrgyz Republic,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,32.855556,0.00,37.666667,36.100000,36.788889,3.933333
Guinea,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,53.477778,0.0,0.000000,0.00,0.000000,56.688889,0.000000,3.211111
Egypt,0.0,15.55,0.0,0.0,0.0,0.0,18.116667,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,2.566667
Guatemala,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,53.818182,0.000000,0.0,0.000000,0.00,56.109091,0.000000,0.000000,2.290909


In [17]:
import altair as alt

# Drop the `change` column
df_pivoted_sorted = df_pivoted_sorted.drop(columns=['change'])

# Melt the dataframe
df_melted = df_pivoted_sorted.reset_index().melt(id_vars='country_name', var_name='year', value_name='value')

# Convert `year` column to datetime
df_melted['year'] = pd.to_datetime(df_melted['year'])

# Create a line chart
chart = alt.Chart(df_melted).mark_line(point=True).encode(
    x=alt.X('year:T', axis=alt.Axis(title='Year', labelAngle=-45)),  # Rotate x-axis labels
    y=alt.Y('value:Q', axis=alt.Axis(title='Poverty Headcount Ratio')),
    color='country_name:N',
    tooltip = ['year', 'value', 'country_name']
).properties(
    title='Poverty Levels by Country and Year'
).interactive()

# Save the chart in json file
chart

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [18]:
# Find the most frequent country
# Count the occurrences of each value in the `country_name` column
country_counts = df['country_name'].value_counts()

# Sort the counts in descending order
country_counts_sorted = country_counts.sort_values(ascending=False)

# Get the country with the most records
most_frequent_country = country_counts_sorted.index[0]

# Print the country with the most records
print(f"The country with the most records is: {most_frequent_country}")

The country with the most records is: Indonesia


In [19]:
# Store Indonesia data
indonesia_data = df[df['country_name'] == 'Indonesia']
indonesia_data

,indicator_name,indicator_code,country_name,country_code,value,year
130,Poverty headcount ratio at national poverty li...,SI.POV.NAHC,Indonesia,IDN_Bali_ID.BA_1513_IDN002,6.890000,2002-01-01
131,Poverty headcount ratio at national poverty li...,SI.POV.NAHC,Indonesia,IDN_Banten_ID.BT_73610_IDN031,9.220000,2002-01-01
132,Poverty headcount ratio at national poverty li...,SI.POV.NAHC,Indonesia,IDN_Bengkulu_ID.BE_1514_IDN003,22.700000,2002-01-01
133,Poverty headcount ratio at national poverty li...,SI.POV.NAHC,Indonesia,IDN_D_I_Yogyakarta_ID.DY_1515_IDN005,20.140000,2002-01-01
134,Poverty headcount ratio at national poverty li...,SI.POV.NAHC,Indonesia,IDN_Dki_Jakarta_ID.DJ_1516_IDN007,3.420000,2002-01-01
...,...,...,...,...,...,...
3625,Poverty headcount ratio at urban poverty line ...,SI.POV.URHC,Indonesia,IDN_Sulawesi_Tenggara_ID.TE_1533_IDN029,4.917453,2013-01-01
3626,Poverty headcount ratio at urban poverty line ...,SI.POV.URHC,Indonesia,IDN_Sulawesi_Utara_ID.SU_73621_IDN024,6.039609,2013-01-01
3627,Poverty headcount ratio at urban poverty line ...,SI.POV.URHC,Indonesia,IDN_Sumatera_Barat_ID.BR_1535_IDN026,6.165176,2013-01-01
3628,Poverty headcount ratio at urban poverty line ...,SI.POV.URHC,Indonesia,IDN_Sumatera_Selatan_ID.SM_73622_IDN027,13.766289,2013-01-01


In [ ]:
/kaggle/input/wb-poverty/wb_subnational_poverty_data.csv